In [1]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import pandas as pd
import os
import numpy as np
from datetime import datetime
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from binance import Client

client = Client()

##################################################
# Lista de parámetros.
##################################################
# Parámetros de interpolación.
###################################
smooth_interval = 60
smooth_exp = 3
###################################
# Parámetros de estrategia.
###################################
rango = 125
std_mult = 250
z_aprox = 0.009

max_points_idx = []
min_points_idx = []
# close_points_max = []
# close_points_min = []
len_maxp = len(max_points_idx)
len_minp = len(min_points_idx)

while True:
    ##################################################################################################################################
    # Descarga y limpieza de datos de 5 días en adelante
    ##################################################################################################################################
    recent_data = pd.read_csv('../csvs/streaming/BTCUSDT.csv')
    recent_data.columns = ['Tiempo','Par','Precio']
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Precio'] = recent_data.Precio.astype(float)
    recent_data = recent_data.reset_index()

    ##################################################
    # Interpolación (suavizado)  del Precio (OPEN) con SciPy y obtención de derivadas con su respectiva interpolación.
    ##################################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Precio,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,3)

    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    # initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[60*45:60*45 + 1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
    # final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango

    initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-60*5:-60*5 + 1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
    final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango

    min_max_sd = pd.DataFrame(columns=['Derivada'])
    lista_min_max = recent_data[initial_date_index:final_date_index][(recent_data[initial_date_index:final_date_index]['TD_Interpolacion'] >= -z_aprox) & (recent_data[initial_date_index:final_date_index]['TD_Interpolacion'] <= z_aprox)]
    indices_min_max = lista_min_max.index.tolist()

    pmin = 0
    pmax = 0

    for j in range(0,len(indices_min_max)):
        PMSD = indices_min_max[j]
        if recent_data.SD_Interpolacion[PMSD:PMSD+1].item() > 0:
            if (abs(recent_data.SD_Interpolacion[PMSD]*1000) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*100) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmax = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                        break
                    elif ((recent_data.SD_Interpolacion[PMSD-i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD-i]*1000)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*100) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD+i]*1000)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*100) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD+i
                except:
                    pass
            try:
                max_points_idx.append(pmax_idx)
            except:
                pass
        elif recent_data.SD_Interpolacion[PMSD:PMSD+1].item() < 0:
            if (abs(recent_data.SD_Interpolacion[PMSD]*1000) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*10) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmin = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                        break
                    elif ((recent_data.SD_Interpolacion[PMSD-i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD-i]*1000) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*100) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD+i]*1000) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*100) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD+i
                except:
                    pass
            try:
                min_points_idx.append(pmin_idx)
            except:
                pass
    max_points_idx = list(set(max_points_idx))
    min_points_idx = list(set(min_points_idx))
    max_points_idx.sort()
    min_points_idx.sort()
    # for lista in [max_points_idx,min_points_idx]:
    #     for i in range(len(lista)-1):
    #         if lista[i+1]-lista[i] == 1:
    #             if lista == max_points_idx:
    #                 close_points_max.append(i+1)
    #             else:
    #                 close_points_min.append(i+1)
    #     if lista == max_points_idx:
    #         for idx in sorted(close_points_max,reverse=True):
    #             del lista[idx]
    #     else:
    #         for idx in sorted(close_points_min,reverse=True):
    #             del lista[idx]
    
    if len(max_points_idx) - len_maxp >= 1:
        print(f'Precio de compra en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de compra: {recent_data.Tiempo[max_points_idx[-1]]}.')
        len_maxp = len(max_points_idx)
    if len(min_points_idx) - len_minp >= 1:
        print(f'Precio de venta en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de venta: {recent_data.Tiempo[max_points_idx[-1]]}.')
        len_minp = len(min_points_idx)

Precio de compra en: 69289.88. Tiempo de compra: 2024-04-07 14:06:05.649000.
Precio de venta en: 69289.88. Tiempo de venta: 2024-04-07 14:06:05.649000.
Precio de venta en: 69289.88. Tiempo de venta: 2024-04-07 14:06:05.649000.
Precio de venta en: 69289.88. Tiempo de venta: 2024-04-07 14:06:05.649000.
Precio de venta en: 69289.88. Tiempo de venta: 2024-04-07 14:06:05.649000.
Precio de venta en: 69289.88. Tiempo de venta: 2024-04-07 14:06:05.649000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de compra en: 69304.48. Tiempo de compra: 2024-04-07 14:06:25.643000.
Precio de

ValueError: time data '2024-04-07 14:26:01' does not match format '%Y-%m-%d %H:%M:%S.%f'